## [Originahttps://poseidon01.ssrn.com/delivery.php?ID=556009073006105088000122086073105078024081022041024036031090115103120083014070085024099002021039108007045065097003086091003112104036001013036115108029089018008107000061062003088073093002097123117025122099081111118125007118076120069029016119120082027&EXT=pdf

In [43]:
import calcbench as cb
import pandas as pd

In [88]:
metrics = ['Assets', 
           'Liabilities',
           'CurrentAssets', 
           'Cash', 
           'ShortTermInvestments', 
           'CurrentLiabilities', 
           'CurrentLongTermDebt', 
           'LongTermDebt',
           'PreferredStockValue',
           'AccountsReceivable',
           'Inventory',
           'PPE',
           'NetIncome',
           'SharesIssued',
           'LoansReceivable',
           'LongTermInvestments',
           'Revenue']

In [52]:
tickers = cb.tickers(index='DJIA')

In [90]:
data = cb.standardized_data(company_identifiers=tickers, metrics=metrics, all_history=True, period_type='annual').fillna(0)

In [84]:
average_total_assets = ((data['Assets'] + data.shift(1)['Assets']) / 2)

In [65]:
#Working Capital
WC = data['CurrentAssets'] - data['Cash'] - data['CurrentLiabilities'] - data['CurrentLongTermDebt'] 

In [66]:
#Net Charge Offs
investments_and_advances = data['LoansReceivable'] # This is probably not totally correct.  The paper says Investments and Advances is DATA 32 from CompuStat
NCO = (data['Assets'] - data['CurrentAssets'] - investments_and_advances) - (data['Liabilities'] - data['CurrentLiabilities'] - data['LongTermDebt'])

In [67]:
FIN = (data['ShortTermInvestments'] + data['LongTermInvestments']) - (data['LongTermDebt'] + data['CurrentLongTermDebt'] + data['PreferredStockValue'])

In [83]:
RSST_Accruals = (WC.diff() + NCO.diff() + FIN.diff()) / average_total_assets

In [40]:
percent_soft_assets = (data['Assets'] - data['PPE']) / data['Assets']

In [60]:
issued_shares = data['SharesIssued'].diff() > 0

In [85]:
change_inventory = data['Inventory'].diff() / average_total_assets

In [86]:
change_receivables = data['AccountsReceivable'] / average_total_assets

In [92]:
change_cash_sales = data['Revenue'] - data['AccountsReceivable'].diff()

In [97]:
ROA = data['NetIncome'] / average_total_assets

In [87]:
intercept = -7.893

In [101]:
m_score = intercept + (.79 * RSST_Accruals) + (2.518 * change_receivables) + (1.191 * change_inventory) + (1.979 * percent_soft_assets) + (.171 * change_cash_sales) + (-.932 * ROA.diff()) + (1.029 * issued_shares)